# Exercise 1: K-Nearest Neighbors (k-NN)



## K-Nearest Neighbors (k-NN)
In this notebook you will implement a K-Nearest Neighbors classifier on the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html).

Recall that the K-Nearest Neighbor classifier does the following:
- During training, the classifier simply memorizes the training data
- During testing, test images are compared to each training image; the predicted label is the majority vote among the K nearest training examples.

After implementing the K-Nearest Neighbor classifier, you will use *cross-validation* to find the best value of K.

The goals of this exercise are to go through a simple example of the data-driven image classification pipeline, and also to practice writing efficient, vectorized code.

## Setup code
Run some setup code for this notebook: Import some useful packages and increase the default figure size.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import statistics
import tensorflow as tf

plt.rcParams['figure.figsize'] = (16.0, 8.0)
plt.rcParams['font.size'] = 16

## Load the CIFAR-10 dataset
We can use the function `tf.keras.datasets.cifar10.load_data()` to load the entire CIFAR-10 dataset as a set of four **NumPy arrays**:

- `x_train` contains all training images (unsigned integers in the range $[0, 255]$)
- `y_train` contains all training labels (integers in the range $[0, 9]$)
- `x_test` contains all test images
- `y_test` contains all test labels

This function automatically downloads the CIFAR-10 dataset the first time you run it.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print('Training set:', )
print('   data shape: \t\t', x_train.shape)
print('   labels shape: \t', y_train.shape)
print('Test set:')
print('   data shape: \t\t', x_test.shape)
print('   labels shape: \t', y_test.shape)

## Visualize the dataset
To give you a sense of the nature of the images in CIFAR-10, this cell visualizes some random examples from the training set.

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7

for y, cls in enumerate(classes):
    idcs = np.flatnonzero(y_train == y)
    idcs = np.random.choice(idcs, samples_per_class, replace=False)
    for i, idx in enumerate(idcs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)

plt.show()

## Subsample the dataset
When implementing machine learning algorithms, it's usually a good idea to use a small sample of the full dataset. This way your code will run much faster, allowing for more interactive and efficient development. Once you are satisfied that you have correctly implemented the algorithm, you can then rerun with the entire dataset.

We will "subsample" (we actually just choose the first n number of samples) the data to use only 500 training examples and 100 test examples.

We also want to normalize the pixel values to be real-valued and range between [0, 1] instead of [0, 255]. 

In [ ]:
def get_CIFAR10_subset(num_train=None, num_test=None):
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

  if num_train is not None:
    x_train = np.divide(x_train[:num_train, :, :, :], 255).astype(np.float32)
    y_train = y_train[:num_train, :]
  if num_test is not None:  
    x_test = np.divide(x_test[:num_test, :, :, :], 255).astype(np.float32)  
    y_test = y_test[:num_test, :]
  return (x_train, y_train), (x_test, y_test)

In [ ]:
num_train = 500
num_test = 100

(x_train, y_train), (x_test, y_test) = get_CIFAR10_subset(num_train, num_test)

print('Training set:', )
print(f'   data: \t shape: {x_train.shape}, {x_train.dtype}')
print(f'   labels: \t shape: {y_train.shape}, {y_train.dtype}')
print('Test set:')
print(f'   data: \t shape: {x_test.shape}, {x_test.dtype}')
print(f'   labels: \t shape: {y_test.shape}, {y_test.dtype}')

## Compute distances: Naive implementation
Now that we have examined and prepared our data, it is time to implement the kNN classifier. We can break the process down into two steps:

1. Compute the (squared Euclidean) distances between all training examples and all test examples
2. Given these distances, for each test example find its k nearest neighbors and have them vote for the label to output

Lets begin with computing the distance matrix between all training and test examples. First we will implement a naive version of the distance computation, using explicit loops over the training and test sets:

**Note: Please try not to use `np.linalg.norm`. Try to write the L2 distance computation from scratch.**

In [ ]:
def compute_distances_two_loops(x_train, x_test):
  """
  Computes the squared Euclidean distance between each element of the training
  set and each element of the test set. Images should be flattened and treated
  as vectors.

  This implementation uses a naive set of nested loops over the training and
  test data.
  
  Inputs:
  - x_train: NumPy array of shape (num_train, H, W, C)
  - x_test: NumPy array of shape (num_test, H, W, C)

  Returns:
  - dists: NumPy array of shape (num_train, num_test) where dists[i, j] 
    is the squared Euclidean distance between the ith training point and the 
    jth test point.
  """
  # Initialize dists to be an array of shape (num_train, num_test) with the
  # same datatype as x_train
  num_train = x_train.shape[0]
  num_test = x_test.shape[0]
  dists = np.zeros((num_train, num_test))
  ##############################################################################
  # TODO: Implement this function using a pair of nested loops over the        #
  # training data and the test data. Note: there is no need to flatten the     #
  # images yet.                                                                #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return dists

After implementing the function above, we can run it to check that it outputs the expected shape:

In [ ]:
dists = compute_distances_two_loops(x_train, x_test)
print('dists has shape: ', dists.shape)

As a visual debugging step, we can visualize the distance matrix, where each row is a test example and each column is a training example.

In [ ]:
plt.imshow(dists, cmap='gray', interpolation='none')
plt.colorbar()
plt.xlabel('test')
plt.ylabel('train')
plt.show()

## Compute distances: Vectorization
Our implementation of the distance computation above is fairly inefficient since it uses nested Python loops over the training and test sets.

When implementing algorithms, it's generally best to avoid loops in Python if possible. Instead it is preferable to implement your computation so that all loops happen inside specialized functions. This will usually be much faster than writing your own loops in Python, since such functions can be internally optimized to iterate efficiently, possibly using multiple threads. This is especially important when using a GPU to accelerate your code.

The process of eliminating explict loops from your code is called **vectorization**. Sometimes it is straighforward to vectorize code originally written with loops; other times vectorizing requires thinking about the problem in a new way. We will use vectorization to improve the speed of our distance computation function.

As a first step toward vectorizing our distance computation, complete the following implementation which uses only a single Python loop over the training data:

In [ ]:
def compute_distances_one_loop(x_train, x_test):
  """
  Computes the squared Euclidean distance between each element of the training
  set and each element of the test set. Images should be flattened and treated
  as vectors.

  This implementation uses only a single loop over the training data.
  
  Inputs:
  - x_train: NumPy array of shape (num_train, H, W, C)
  - x_test: NumPy array of shape (num_test, H, W, C)

  Returns:
  - dists: NumPy array of shape (num_train, num_test) where dists[i, j] 
    is the squared Euclidean distance between the ith training point and the 
    jth test point.
  """
  # Initialize dists to be an array of shape (num_train, num_test) with the
  # same datatype as x_train
  num_train = x_train.shape[0]
  num_test = x_test.shape[0]
  dists = np.zeros((num_train, num_test))
  ##############################################################################
  # TODO: Implement this function using a single loop over the training data.  #
  # Note: there is no need to flatten the images yet.                          #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return dists

We can check the correctness of our one-loop implementation by comparing it with our two-loop implementation on some randomly generated data.

Note that we do the comparison with 64-bit floating points for increased numeric precision.

In [ ]:
np.random.seed(0)
x_train_rand = np.random.standard_normal(size=(100, 16, 16, 3)).astype(np.float64)
x_test_rand = np.random.standard_normal(size=(100, 16, 16, 3)).astype(np.float64)

dists_one = compute_distances_one_loop(x_train_rand, x_test_rand)
dists_two = compute_distances_two_loops(x_train_rand, x_test_rand)
difference = np.sqrt(np.sum((dists_one - dists_two)**2))
print('Difference: ', difference)
if difference < 1e-4:
  print('Good! The distance matrices match')
else:
  print('Uh-oh! The distance matrices are different')

Now implement a fully vectorized version of the distance computation function
that does not use any python loops.

In [ ]:
def compute_distances_no_loops(x_train, x_test):
  """
  Computes the squared Euclidean distance between each element of the training
  set and each element of the test set. Images should be flattened and treated
  as vectors.

  This implementation should not use any Python loops. For memory-efficiency,
  it also should not create any large intermediate arrays; in particular you
  should not create any intermediate arrays with O(num_train*num_test)
  elements.
  
  Inputs:
  - x_train: NumPy array of shape (num_train, H, W, C)
  - x_test: NumPy array of shape (num_test, H, W, C)

  Returns:
  - dists: NumPy array of shape (num_train, num_test) where dists[i, j] 
    is the squared Euclidean distance between the ith training point and the 
    jth test point.
  """
  # Initialize dists to be an array of shape (num_train, num_test) with the
  # same datatype as x_train
  num_train = x_train.shape[0]
  num_test = x_test.shape[0]
  dists = np.zeros((num_train, num_test))
  #################################################################################################################
  # TODO: Implement this function using no loops over the training data and the test data.                        #
  #                                                                                                               #
  # HINT: Try to formulate the Euclidean distance using two broadcast sums                                        #
  #       and a matrix multiply.                                                                                  #
  #       https://stackoverflow.com/questions/27948363/numpy-broadcast-to-perform-euclidean-distance-vectorized.  #
  #################################################################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return dists

As before, we can check the correctness of our implementation by comparing the fully vectorized version against the original naive version:

In [ ]:
np.random.seed(0)
x_train_rand = np.random.standard_normal(size=(100, 16, 16, 3)).astype(np.float64)
x_test_rand = np.random.standard_normal(size=(100, 16, 16, 3)).astype(np.float64)

dists_none = compute_distances_no_loops(x_train_rand, x_test_rand)
dists_two = compute_distances_two_loops(x_train_rand, x_test_rand)
difference = np.sqrt(np.sum((dists_one - dists_two)**2))
print('Difference: ', difference)
if difference < 1e-4:
  print('Good! The distance matrices match')
else:
  print('Uh-oh! The distance matrices are different')

We can now compare the speed of our three implementations. If you've implemented everything properly, the one-loop implementation should take less than 4 seconds to run, and the fully vectorized implementation should take less than 0.1 seconds to run.

In [ ]:
import time

def timeit(f, *args):
  tic = time.time()
  f(*args) 
  toc = time.time()
  return toc - tic

np.random.seed(0)
x_train_rand = np.random.standard_normal(size=(500, 32, 32, 3)).astype(np.float64)
x_test_rand = np.random.standard_normal(size=(500, 32, 32, 3)).astype(np.float64)

two_loop_time = timeit(compute_distances_two_loops, x_train_rand, x_test_rand)
print('Two loop version took %.2f seconds' % two_loop_time)

one_loop_time = timeit(compute_distances_one_loop, x_train_rand, x_test_rand)
speedup = two_loop_time / one_loop_time
print('One loop version took %.2f seconds (%.1fX speedup)'
      % (one_loop_time, speedup))

no_loop_time = timeit(compute_distances_no_loops, x_train_rand, x_test_rand)
speedup = two_loop_time / no_loop_time
print('No loop version took %.2f seconds (%.1fX speedup)'
      % (no_loop_time, speedup))

## Predict labels
Now that we have a method for computing distances between training and test examples, we need to implement a function that uses those distances together with the training labels to predict labels for test samples.

Complete the implementation of the function below:

In [ ]:
def predict_labels(dists, y_train, k=1):
  """
  Given distances between all pairs of training and test samples, predict a
  label for each test sample by taking a majority vote among its k nearest
  neighbors in the training set.
 
  In the event of a tie, this function should return the smaller label. For
  example, if k=5 and the 5 nearest neighbors to a test example have labels
  [1, 2, 1, 2, 3] then there is a tie between 1 and 2 (each have 2 votes), so
  we should return 1 since it is the smaller label.

  Inputs:
  - dists: NumPy array of shape (num_train, num_test) where dists[i, j] is the
    squared Euclidean distance between the ith training point and the jth test
    point.
  - y_train: NumPy array of shape (y_train,) giving labels for all training
    samples. Each label is an integer in the range [0, num_classes - 1]
  - k: The number of nearest neighbors to use for classification.

  Returns:
  - y_pred: A NumPy array of shape (num_test,) and type int64 giving predicted 
    labels for the test data, where y_pred[j] is the predicted label for the jth 
    test example. Each label should be an integer in the range 
    [0, num_classes - 1].
  """
  num_train, num_test = dists.shape
  y_pred = np.zeros(num_test, dtype=np.int64)
  ##############################################################################
  # TODO: Implement this function. You may use an explicit loop over the test  #
  # samples.                                                                   #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                             END OF YOUR CODE                               #
  ##############################################################################
  return y_pred

Now we have implemented all the required functionality for the K-Nearest Neighbor classifier. We can define a simple object to encapsulate the classifier:

In [ ]:
class KnnClassifier:
  def __init__(self, x_train, y_train):
    """
    Create a new K-Nearest Neighbor classifier with the specified training data.
    In the initializer we simply memorize the provided training data.

    Inputs:
    - x_train: NumPy array of shape (num_train, H, W, C) giving training data
    - y_train: NumPy array of shape (num_train,) giving training labels
    """
    self.x_train = x_train
    self.y_train = y_train
  
  def predict(self, x_test, k=1):
    """
    Make predictions using the classifier.
   
    Inputs:
    - x_test: NumPy array of shape (num_test, H, W, C) giving test samples
    - k: The number of neighbors to use for predictions
  
    Returns:
    - y_test_pred: NumPy array of shape (num_test,) giving predicted labels
      for the test samples.
    """
    dists = compute_distances_no_loops(self.x_train, x_test)
    y_test_pred = predict_labels(dists, self.y_train, k=k)
    return y_test_pred
  
  def check_accuracy(self, x_test, y_test, k=1, quiet=False):
    """
    Utility method for checking the accuracy of this classifier on test data.
    Returns the accuracy of the classifier on the test data, and also prints a
    message giving the accuracy.

    Inputs:
    - x_test: NumPy array of shape (num_test, H, W, C) giving test samples
    - y_test: NumPy array of shape (num_test,) giving test labels
    - k: The number of neighbors to use for prediction
    - quiet: If True, don't print a message.
  
    Returns:
    - accuracy: Accuracy of this classifier on the test data, as a percentage.
      Python float in the range [0, 100]
    """
    y_test_pred = self.predict(x_test, k=k)
    num_samples = x_test.shape[0]
    num_correct = np.sum(y_test.reshape(-1) == y_test_pred)
    accuracy = 100.0 * num_correct / num_samples
    msg = (f'Got {num_correct} / {num_samples} correct; '
           f'accuracy is {accuracy:.2f}%')
    if not quiet:
      print(msg)
    return accuracy

Now lets put everything together and test our K-NN clasifier on a subset of CIFAR-10, using k=1:

If you've implemented everything correctly you should see an accuracy of about 27%.

In [ ]:
num_train = 5000
num_test = 500

(x_train, y_train), (x_test, y_test) = get_CIFAR10_subset(num_train, num_test)

classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=1)

Now lets increase to k=5. You should see a slightly higher accuracy than k=1:

In [ ]:
classifier.check_accuracy(x_test, y_test, k=5)

## Cross-validation
We have now implemented the full k-Nearest Neighbor classifier, but the choice of $k=5$ was arbitrary. We will use **cross-validation** to set this hyperparameter in a more principled manner.

Implement the following function to run cross-validation:

In [ ]:
def knn_cross_validate(x_train, y_train, num_folds=5, k_choices=None):
  """
  Perform cross-validation for KnnClassifier.

  Inputs:
  - x_train: NumPy array of shape (num_train, H, W, C) giving all training data
  - y_train: NumPy array of shape (num_train,) giving labels for training data
  - num_folds: Integer giving the number of folds to use
  - k_choices: List of integers giving the values of k to try
 
  Returns:
  - k_to_accuracies: Dictionary mapping values of k to lists, where
    k_to_accuracies[k][i] is the accuracy on the ith fold of a KnnClassifier
    that uses k nearest neighbors.
  """
  if k_choices is None:
    # Use default values
    k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

  if x_train.shape[0] % num_folds > 0:
    raise ValueError(
        'Number of training examples not evenly divisible by number of folds.')

  # First we divide the training data into num_folds equally-sized folds.
  x_train_folds = []
  y_train_folds = []
  ##############################################################################
  # TODO: Split the training data and images into folds. After splitting,      #
  # x_train_folds and y_train_folds should be lists of length num_folds, where #
  # y_train_folds[i] is the label vector for images in x_train_folds[i].       #
  # Hint: np.split()                                                           #
  ##############################################################################
  # Replace "pass" statement with your code
  pass
  ##############################################################################
  #                            END OF YOUR CODE                                #
  ##############################################################################
  
  # A dictionary holding the accuracies for different values of k that we find
  # when running cross-validation. After running cross-validation,
  # k_to_accuracies[k] should be a list of length num_folds giving the different
  # accuracies we found when trying KnnClassifiers that use k neighbors.
  k_to_accuracies = {}

  ##############################################################################
  # TODO: Perform cross-validation to find the best value of k. For each value #
  # of k in k_choices, run the k-nearest-neighbor algorithm num_folds times;   #
  # in each case you'll use all but one fold as training data, and use the     #
  # last fold as a validation set. Store the accuracies for all folds and all  #
  # values in k in k_to_accuracies.                                            #
  #                                                                            #
  # HINT: np.concatenate                                                       #
  ##############################################################################
  # Replace "pass" statement with your code
  # Perform k-fold cross validation to find the best value of k
  pass
  ##############################################################################
  #                            END OF YOUR CODE                                #
  ##############################################################################

  return k_to_accuracies

Now we'll run your cross-validation function:

In [ ]:
num_train = 5000
num_test = 500

(x_train, y_train), (x_test, y_test) = get_CIFAR10_subset(num_train, num_test)

k_to_accuracies = knn_cross_validate(x_train, y_train, num_folds=5)

for k, accs in sorted(k_to_accuracies.items()):
  print('k = %d got accuracies: %r' % (k, accs))

In [ ]:
ks, means, stds = [], [], []
for k, accs in sorted(k_to_accuracies.items()):
  plt.scatter([k] * len(accs), accs, color='g')
  ks.append(k)
  means.append(statistics.mean(accs))
  stds.append(statistics.stdev(accs))
plt.errorbar(ks, means, yerr=stds)
plt.xlabel('k')
plt.ylabel('Cross-validation accuracy')
plt.title('Cross-validation on k')
plt.show()

Now we can use the results of cross-validation to select the best value for k, and rerun the classifier on our full 5000 set of training examples.

You should get an accuracy above 28%.

In [ ]:
best_k = 1
##############################################################################
# TODO: Use the results of cross-validation stored in k_to_accuracies to     #
# choose the value of k, and store the result in best_k. You should choose   #
# the value of k that has the highest mean accuracy accross all folds.       #
##############################################################################
# Replace "pass" statement with your code
pass
##############################################################################
#                            END OF YOUR CODE                                #
##############################################################################
    
print('Best k is ', best_k)
classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=best_k)

Finally, we can use our chosen value of k to run on the entire training and test sets.

This may take a while to run, since the full training and test sets have 50k and 10k examples respectively. You should get an accuracy above 33%.

**Run this only once!**

In [ ]:
(x_train, y_train), (x_test, y_test) = get_CIFAR10_subset()
classifier = KnnClassifier(x_train, y_train)
classifier.check_accuracy(x_test, y_test, k=best_k)